# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 08:31:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   49C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 LG Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_lg.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2020010204,"LG, 롤러블·월페이퍼…`올레드TV 혁신`",http://news.mk.co.kr/newsRead.php?no=2411&year...,◆ 2020 재계가 뛴다 ◆\r\n\n\n구광모 LG그룹 회장(가운데)이 서울 강서...,2020-01-02,4,321000,321000,313500,314000,139496,-0.011024,-1,0,재계 구광모 그룹 회장 가운데 서울 강서구 마곡 스파크 방문 연구원 투명 플렉시블 ...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2020 = pos_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <장점> & <강점> : 0.5735523700714111
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2020-01-02 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2020-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2020-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2020-01-02 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2020-01-02 Cosine Similarity between <개선> & <방안> : 0.5635414123535156
***2020-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2020-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2020-01-02 Cosine Similarity between <실현> & <구현> : 0.5969869494438171
***2020-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <증

In [11]:
pos_lexicon_2020

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,장점,강점,0.573552,1
1,2020-01-02,1,증대,확대,0.519324,1
2,2020-01-02,1,증대,급증,0.505104,1
3,2020-01-02,1,추진,계획,0.520542,2
4,2020-01-02,1,추진,추진,1.000000,1
...,...,...,...,...,...,...
11172,2021-01-04,2055,증대,증가,0.694643,2
11173,2021-01-04,2055,증대,확대,0.519324,1
11174,2021-01-04,2055,증대,급증,0.505104,1
11175,2021-01-04,2055,추진,추진,1.000000,1


In [12]:
pos_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/pos_lg_2020.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2020['news_word'])))

a

['언택트주',
 '업황',
 '의미',
 '판매승인',
 '설치량',
 '나정환',
 '셀리백스',
 '코슈메슈티컬',
 '트윈데믹',
 '가장',
 '주요한',
 '거버넌스위',
 '제미메트',
 '폴딩드레스룸',
 '엔드플레이트',
 '시너지',
 '재원임',
 '강세',
 '골든써밋',
 '향상',
 '활황',
 '액티브펀드',
 '추천',
 '이홍락',
 '코세스',
 '루브리컨츠',
 '횡보할',
 '중인',
 '마련',
 '랙티브',
 '듀비에',
 '제놀루션제놀루션',
 '루시드모터스',
 '충남소방',
 '실패',
 '기록',
 '트라젠타',
 '감소',
 '언택트',
 '컨설팅펌',
 '이엔드디이엔드디',
 '장단점',
 '케이에스',
 '문제',
 '송승봉',
 '센티먼트',
 '맞소송',
 '강력',
 '사하나',
 '겹악재',
 '계획',
 '안현식',
 '오벤터스',
 '프론텍',
 '적극',
 '중요',
 '렌털업체',
 '수펙스홀',
 '단점',
 '밑돈',
 '침체',
 '폭락',
 '증대',
 '활기',
 '이점',
 '확립',
 '훈풍',
 '증가',
 '밸트로핀',
 '불황',
 '추진',
 '해법',
 '저하',
 '켁스트',
 '확대',
 '급락',
 '블루써밋',
 '티에닐메틸',
 '리레이팅',
 '엠비엔',
 '쎄미켐동진쎄미켐',
 '강점',
 '확충',
 '프로그램매매',
 '전개',
 '웃돈',
 '박기효',
 '류지민',
 '촉진',
 '견조할',
 '요구',
 '창저우시',
 '추동훈',
 '인멸된',
 '임방현',
 '광풍',
 '검토',
 '텍이엠텍',
 '흐트러지',
 '우세',
 '맥쿼리인프라',
 '증진',
 '기가와트',
 '츠코프',
 '우호',
 '들보',
 '박유악',
 '이노톡스',
 '본격',
 '도대사',
 '손효숙씨',
 '텍드림텍',
 '인플렉트',
 '펀인터렉티브',
 '방안',
 '난펑그룹',
 '스제약위더스',
 '케이티엑스',
 '할그렌',
 '툴리눔',
 '페르다나쿠',
 '

In [14]:
b = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['추천',
 '툴리눔',
 '쎄미켐동진쎄미켐',
 '불황',
 '추진',
 '단점',
 '밑돈',
 '최고점',
 '실패',
 '우세',
 '약세',
 '프리사']

In [15]:
c = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['쎄미켐동진쎄미켐',
 '최고점',
 '맞소송',
 '재택명령',
 '웃돌',
 '트윈데믹',
 '프리사',
 '오벤터스',
 '적극',
 '단점',
 '밑돈',
 '우세',
 '츠코프',
 '추천',
 '것이므',
 '증가',
 '제놀루션제놀루션',
 '툴리눔',
 '불황',
 '추진',
 '실패',
 '약세',
 '감소']